<a href="https://colab.research.google.com/github/lucasreis95/world-surf-league-data/blob/main/notebooks/etl/02_bronze_events_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libs
import pandas as pd
from datetime import date
import pandas_gbq
import numpy as np
import requests
from bs4 import BeautifulSoup

In [ ]:
# get main url (don't include year)
url = 'https://www.worldsurfleague.com/athletes/tour/mct?year='

In [ ]:
# set empty dfs that will be filled
year_list_to_df = []
event_name_list = []
event_order_list = []
# loop throught years to get all events
year_list = [i for i in range(2010, 2025) if i != 2020]
for year in year_list:
  # convert to string to concatenate with url
  year_str = str(year)
  # make request
  r = requests.get(url + year_str).text
  soup = BeautifulSoup(r, 'html.parser')
  # get html with all events of a year
  all_events_html = soup.find_all('span', attrs = {'class':'tooltip-item'})
  # we use a way greater range than the maximum events per year
  # to guarantee that we will include all events
  for i in range(18):
    try:
      # get event name
      event_name = all_events_html[i]['data-tooltip'].split('\\">')[2].split('<\\/')[0]
      # append auxiliar information to empty lists
      event_name_list.append(event_name)
      event_order_list.append(i)
      year_list_to_df.append(year)
    # as we don't have a fixed number of events per year, we need to use this exception
    # and element 'data-tooltip' is not provided for future events
    except:
      pass
    try:
      # get event name
      event_name = all_events_html[i]['data-tooltip'].split('\\">')[1].split('<\\/')[0] # 2023 and 2024 rankings are slight different
      # append auxiliar information to empty lists
      event_name_list.append(event_name)
      event_order_list.append(i)
      year_list_to_df.append(year)
    except:
      pass
  print('Events from ', year, ' has been colected.')

# create df
df_events = pd.DataFrame(
                        {'season_year': year_list_to_df,
                         'event_order': event_order_list,
                         'event_name': event_name_list
                        })

#remove elemetns from df
df_events = df_events[~df_events['event_name'].str.contains('<span')]
# sum 1 to event_order to start from 1 instead of 0
df_events['event_order'] = df_events['event_order'] + 1

Events from  2010  has been colected.
Events from  2011  has been colected.
Events from  2012  has been colected.
Events from  2013  has been colected.
Events from  2014  has been colected.
Events from  2015  has been colected.
Events from  2016  has been colected.
Events from  2017  has been colected.
Events from  2018  has been colected.
Events from  2019  has been colected.
Events from  2021  has been colected.
Events from  2022  has been colected.
Events from  2023  has been colected.
Events from  2024  has been colected.


In [ ]:
# write raw table in big query
pandas_gbq.to_gbq(
                  dataframe = df_events,
                  destination_table = 'wsl-data-397017.01_bronze.wsl_events_scrap',
                  project_id = 'wsl-data-397017',
                  if_exists = 'replace'
                  )

100%|██████████| 1/1 [00:00<00:00, 7943.76it/s]
